In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import random_split
import random
import os
#import Image
from PIL import Image


device = torch.device("cpu")

classes = ['3d','c0','c1','c10','c2','c3','c4','c5','c6','c7','c8','c9','e1','e2','ent','ex','exb','i','m','meet','mod','r','sew','sv','tech','wm1','wm2']





In [3]:

def load_resnet50_finetuned():
    # Load a pre-trained ResNet50 model
    model_resnet50 = models.resnet50(pretrained=True)

    # Unfreeze some of the layers for fine-tuning
    for name, child in model_resnet50.named_children():
        if name in ['layer3', 'layer4']:
            for param in child.parameters():
                param.requires_grad = True
        else:
            for param in child.parameters():
                param.requires_grad = False

    # Modify the final layer for  len(dataset.classes) classes
    num_ftrs = model_resnet50.fc.in_features
    model_resnet50.fc = nn.Linear(num_ftrs,  len(classes))

    model_resnet50 = model_resnet50.to(device)

    # Define loss function and optimizer for ResNet50
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_resnet50.parameters()), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    return model_resnet50, optimizer, criterion


def load_resnet101():
    # Load a pre-trained ResNet101 model
    model_resnet101 = models.resnet101(pretrained=True)

    # Modify the final layer for  len(dataset.classes) classes
    num_ftrs = model_resnet101.fc.in_features
    model_resnet101.fc = nn.Linear(num_ftrs,  len(classes))

    model_resnet101 = model_resnet101.to(device)

    # Define loss function and optimizer for ResNet101
    optimizer = optim.Adam(model_resnet101.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    return model_resnet101, optimizer, criterion

from PIL import Image
import torchvision.transforms as transforms

chosen_model = load_resnet101()[0]
#load from state dict
chosen_model.load_state_dict(torch.load('./models/FC_Res101_simple/epoch_8.pth', map_location=torch.device('cpu')))

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [6]:


def do_inference(image):
    # Function to preprocess the image
    def preprocess_image(image_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        #directly take in image file in variable
        # image = Image.open(image)
        
        image = Image.open(image_path)
        image = Image.open(image_path).convert('RGB')  # Convert to RGB
        image = transform(image).unsqueeze(0)  # Add batch dimension
        return image

    def predict_image(model, image_path, class_names):
        image = preprocess_image(image_path)
        image = image.to(device)

        model.eval()
        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
            predicted_class = class_names[predicted[0].item()]

        return predicted_class

    class_names = classes
    preprocess_image(image)
    p = predict_image(chosen_model, image, class_names)
    # print(p)
    return p


In [7]:
do_inference('./frames/2.jpg')

'c2'

In [1]:
#check:
# Completing the edges with distances and labels

import networkx as nx

# Define the edges with their distances and labels
edges_with_distances_and_labels = {
    ('WM1', 'WM2'): {'distance': 9, 'label': 90},
    ('WM2', 'WM1'): {'distance': 9, 'label': 270},
    ('EX', 'C0'): {'distance': 6, 'label': 90},
    ('C0', 'EX'): {'distance': 6, 'label': 270},
    ('C0', 'C5'): {'distance': 5, 'label': 90},
    ('C5', 'C0'): {'distance': 5, 'label': 270},
    ('C1', 'C2'): {'distance': 7, 'label': 90},
    ('C2', 'C1'): {'distance': 7, 'label': 270},
    ('C5', 'SV'): {'distance': 3, 'label': 90},
    ('SV', 'C5'): {'distance': 3, 'label': 270},
    ('E1', 'E2'): {'distance': 7, 'label': 90},
    ('E2', 'E1'): {'distance': 7, 'label': 270},
    ('E2', 'M'): {'distance': 6, 'label': 90},
    ('M', 'E2'): {'distance': 6, 'label': 270},
    ('I', 'C6'): {'distance': 3, 'label': 90},
    ('C6', 'I'): {'distance': 3, 'label': 270},
    ('C2', 'C3'): {'distance': 4, 'label': 90},
    ('C3', 'C2'): {'distance': 4, 'label': 270},
    ('C3', 'C4'): {'distance': 5, 'label': 90},
    ('C4', 'C3'): {'distance': 5, 'label': 270},
    ('EXB', 'C9'): {'distance': 8, 'label': 90},
    ('C9', 'EXB'): {'distance': 8, 'label': 270},
    ('C10', 'ENT'): {'distance': 4, 'label': 90},
    ('ENT', 'C10'): {'distance': 4, 'label': 270},
    ('C7', 'MEET'): {'distance': 4, 'label': 90},
    ('MEET', 'C7'): {'distance': 4, 'label': 270},
    ('R', '3D'): {'distance': 5, 'label': 90},
    ('3D', 'R'): {'distance': 5, 'label': 270},
    ('C0', 'WM2'): {'distance': 8, 'label': 0},
    ('WM2', 'C0'): {'distance': 8, 'label': 180},
    ('MOD', 'C1'): {'distance': 8, 'label': 0},
    ('C1', 'MOD'): {'distance': 8, 'label': 180},
    ('C5', 'E1'): {'distance': 8, 'label': 0},
    ('E1', 'C5'): {'distance': 8, 'label': 180},
    ('C1', 'C5'): {'distance': 4, 'label': 0},
    ('C5', 'C1'): {'distance': 4, 'label': 180},
    ('C8', 'C2'): {'distance': 8, 'label': 0},
    ('C2', 'C8'): {'distance': 8, 'label': 180},
    ('C2', 'C6'): {'distance': 4, 'label': 0},
    ('C6', 'C2'): {'distance': 4, 'label': 180},
    ('C6', 'E2'): {'distance': 8, 'label': 0},
    ('E2', 'C6'): {'distance': 8, 'label': 180},
    ('C3', 'TEC'): {'distance': 4, 'label': 0},
    ('TEC', 'C3'): {'distance': 4, 'label': 180},
    ('R', 'C3'): {'distance': 5, 'label': 0},
    ('C3', 'R'): {'distance': 5, 'label': 180},
    ('C4', 'SEW'): {'distance': 4, 'label': 0},
    ('SEW', 'C4'): {'distance': 4, 'label': 180},
    ('3D', 'C4'): {'distance': 5, 'label': 0},
    ('C4', '3D'): {'distance': 5, 'label': 180},
    ('EXB', '3D'): {'distance': 4, 'label': 0},
    ('3D', 'EXB'): {'distance': 4, 'label': 180},
    ('C7', 'C10'): {'distance': 5, 'label': 0},
    ('C10', 'C7'): {'distance': 5, 'label': 180},
    ('C9', 'C7'): {'distance': 4, 'label': 0},
    ('C7', 'C9'): {'distance': 4, 'label': 180},
    ('C4', 'C10'): {'distance': 3, 'label': 90},
    ('C10', 'C4'): {'distance': 3, 'label': 270}
}


# Create a directed graph
G_directional_distances = nx.DiGraph()

# Add edges to the graph with distances as weights
for edge, attributes in edges_with_distances_and_labels.items():
    G_directional_distances.add_edge(edge[0], edge[1], weight=attributes['distance'], label=attributes['label'])

# Function to find the shortest path with Dijkstra's algorithm considering distance
def find_shortest_path_dijkstra(graph, start_node, end_node):
    try:
        # Compute the shortest path using Dijkstra's algorithm
        path = nx.dijkstra_path(graph, source=start_node, target=end_node, weight='weight')
        for i in range(len(path)-1):
            edge = (path[i], path[i+1])
            direction = graph.edges[edge]['label']
            distance = graph.edges[edge]['weight']
        return path
    except (nx.NetworkXNoPath, KeyError):
        # Return an empty list if no path exists or if the nodes are not in the graph
        return []



def find_shortest_path_bfs(graph, start_node, end_node):

    try:
        path = find_shortest_path_dijkstra(graph, start_node, end_node)
        return path
    except (nx.NetworkXNoPath, KeyError):
        # Return an empty list if no path exists or if the nodes are not in the graph
        return []

# Example usage of the function
# Find the shortest path from 'CAF' to 'ENT'
path_caf_to_ent = find_shortest_path_bfs(G_directional_distances, 'ENT', 'WM1')
path_caf_to_ent

['ENT', 'C10', 'C4', 'C3', 'C2', 'C1', 'C5', 'C0', 'WM2', 'WM1']

In [2]:
shift = 45
def get_direction_and_path(graph, start_node, end_node, alpha_rotation):
    """
    Get the direction and path from start_node to end_node in a graph.

    :param graph: NetworkX graph
    :param start_node: starting node
    :param end_node: ending node
    :param alpha_rotation: current alpha rotation with respect to true North
    :return: tuple (path, directions)
    """

    def calculate_direction(edge, current_alpha):
        """
        Calculate the direction to face for a given edge, adjusting for the current alpha rotation.
        The graph directions are 45 degrees to the right of true north and are clockwise.
        """
        graph_direction = graph.edges[edge]['label']
        adjusted_direction = (graph_direction - current_alpha + 360 + shift) % 360
        return adjusted_direction

    path = find_shortest_path_bfs(graph, start_node, end_node)
    directions = []


    edge = (path[0], path[1])
    direction = calculate_direction(edge, alpha_rotation)


    return path, direction






In [5]:
def get_relative_direction(angle):
    """
    Convert an angle to a relative direction (F, L, B, R).

    :param angle: Angle in degrees, where 0 degrees is the direction you're facing
    :return: A string representing the relative direction ('F', 'L', 'B', 'R')
    """
    if 315 <= angle < 360 or 0 <= angle < 45:
        return 'F'  # Forward
    elif 45 <= angle < 135:
        return 'R'  # Right
    elif 135 <= angle < 225:
        return 'B'  # Backward
    elif 225 <= angle < 315:
        return 'L'  # Left
    else:
        return 'Unknown'  # For unexpected cases

# Example usage
path, angle = get_direction_and_path(G_directional_distances, 'C1', 'C5', 200)  # Replace 0 with your alpha rotation
print("Path:", path)


direction = get_relative_direction(angle)
print("Relative Direction:", direction)


Path: ['C1', 'C5']
Relative Direction: B
